In [16]:
import pandas as pd
# train = pd.read_csv('data/train.csv')
train = pd.read_csv('train_preprocessed.csv')
train = train.drop(columns=['TypeofContact','Occupation','NumberOfPersonVisiting','NumberOfTrips','OwnCar','NumberOfChildrenVisiting'])
train.head()

,Age,CityTier,DurationOfPitch,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken
0,0.232558,1,0.277778,2,4.0,0,3.0,1,0,1,1,0.198448,0
1,0.372093,3,0.000000,1,4.0,1,4.0,2,1,5,2,0.190411,1
2,0.627907,1,0.000000,2,3.0,1,4.0,1,0,4,2,0.231619,0
3,0.255814,1,0.194444,2,5.0,0,4.0,1,0,4,1,0.207560,1
4,0.558140,3,0.166667,2,3.0,1,3.0,0,0,3,2,0.193565,0


In [17]:
from pycaret.datasets import get_data 
from pycaret.classification import *
from pycaret.utils import check_metric

In [18]:
# setup으로 init
clf = setup(data=train, target='ProdTaken', use_gpu=True)

,Description,Value
0,session_id,6507
1,Target,ProdTaken
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(1955, 13)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [19]:
# ML 모델 성능 비교
top5 = compare_models(sort='Accuracy', n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8750,0.8583,0.4823,0.7623,0.5856,0.5169,0.5383,0.4030
rf,Random Forest Classifier,0.8721,0.8637,0.3872,0.8354,0.5227,0.4614,0.5095,0.4260
lightgbm,Light Gradient Boosting Machine,0.8720,0.8425,0.4743,0.7539,0.5757,0.5057,0.5279,0.7250
lr,Logistic Regression,0.8640,0.7988,0.3626,0.7954,0.4917,0.4267,0.4733,0.0540
lda,Linear Discriminant Analysis,0.8625,0.8016,0.4062,0.7328,0.5167,0.4455,0.4745,0.0130
gbc,Gradient Boosting Classifier,0.8582,0.8139,0.3795,0.7300,0.4906,0.4193,0.4528,0.1570
ridge,Ridge Classifier,0.8523,0.0000,0.2406,0.8667,0.3748,0.3203,0.4068,0.0080
ada,Ada Boost Classifier,0.8523,0.7778,0.3626,0.6921,0.4728,0.3973,0.4267,0.0980
knn,K Neighbors Classifier,0.8399,0.7849,0.2995,0.6373,0.4033,0.3264,0.3586,0.3240
dt,Decision Tree Classifier,0.8216,0.7102,0.5334,0.5226,0.5231,0.4142,0.4172,0.0120


In [20]:
top5

[ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=6507, verbose=0,
                      warm_start=False),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=-

In [21]:
# 모델 튜닝, blending
tuned_top5 = [tune_model(i) for i in top5]
blender_top5 = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8540,0.7757,0.2800,0.7778,0.4118,0.3489,0.4087
1,0.8467,0.8675,0.3200,0.6667,0.4324,0.3562,0.3884
2,0.8905,0.8657,0.4800,0.8571,0.6154,0.5574,0.5893
3,0.8759,0.7821,0.4800,0.7500,0.5854,0.5165,0.5343
4,0.8613,0.8414,0.4000,0.7143,0.5128,0.4394,0.4645
5,0.8759,0.8541,0.4615,0.8000,0.5854,0.5185,0.5457
6,0.8467,0.7921,0.3462,0.6923,0.4615,0.3835,0.4149
7,0.8613,0.8257,0.3077,0.8889,0.4571,0.3984,0.4728
8,0.8529,0.7279,0.3200,0.7273,0.4444,0.3741,0.4162


In [ ]:
blender_top5

In [22]:
# 모델 최종 확정, 전체 데이터 학습
final_model = finalize_model(blender_top5)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

In [23]:
# test 데이터 불러오기
# test = pd.read_csv('data/test.csv')
test = pd.read_csv('test_preprocessed.csv')
test  = test.drop(columns=['TypeofContact','Occupation','NumberOfPersonVisiting','NumberOfTrips','OwnCar','NumberOfChildrenVisiting'])

In [24]:
# 테스트셋 predict
prediction = predict_model(final_model, data=test)

In [25]:
prediction.head()

,Age,CityTier,DurationOfPitch,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Label,Score
0,0.325581,3,0.000000,2,5.000000,1,3.0,1,0,2,2,0.191118,0,0.8521
1,0.651163,2,0.305556,2,3.701827,1,4.0,1,1,5,2,0.194732,0,0.7589
2,0.441860,3,0.611111,2,4.000000,1,3.0,1,0,5,2,0.208174,0,0.8934
3,0.581395,1,1.000000,2,6.000000,1,3.0,3,0,3,2,0.224718,0,0.8689
4,0.162791,3,0.194444,1,4.000000,0,4.0,3,1,4,1,0.213764,1,0.7462


In [ ]:
# prediction.to_csv('prediction.csv',index = False)

In [26]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['ProdTaken'] = prediction['Label']
sample_submission.to_csv('submission.csv',index = False)